In [3]:
import random
import numpy as np

In [4]:
# Hyperparameters
learning_rate = 0.01

**Perceptron Class**

Here we create a class for the Perceptron that we may use to instatiate any number of Perceptron instances.

In [69]:
class Perceptron:
    
    def __init__(self, num_inputs, act='sigmoid'):
        self.weights = []
        self.num_inputs = num_inputs
        self.act = act # define activation function with sigmoid being the default
        for x in range(0, num_inputs):
            self.weights.append(random.random() * 2 - 1)
        print(self.weights)
            
    def get_weights(self):
        return self.weights
        
    def feed_forward(self, inputs):
        self.inputs = inputs
        sum = 0
        
        # multiply inputs by weights and sum them
        for i in range(0, self.num_inputs):
            sum += self.weights[i] * inputs[i]
            
        # 'activate' the sum and get the derivative
        self.output, self.output_prime = self.activate(sum)
        return self.output
    
    def activate(self, x):
        if (self.act == 'sigmoid'):
            activation = self.sigmoid(x)
            activation_prime = activation * (1 - activation)
        else:
            activation = self.step(x)
            activation_prime = 1 # use 1 since step activation is not differentiable
        return activation, activation_prime
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x))
    
    def step(self, x):
        if x > 0:
            return 1
        return 0
    
    def backward_pass(self, error):
        back_error = [] # each element in list represent amount of error to send backward along that connection
        for i in range(0, self.num_inputs):
            back_error.append(error * self.output_prime * self.weights[i])
            self.weights[i] -= error * self.output_prime * self.inputs[i] * learning_rate
        return back_error

**Single Perceptron to classify a point as being above or below a line**

Begin with a line going through the origin.

In [77]:
def line(x):
    return 0.5 * x

In [78]:
p = Perceptron(2)
weights = p.get_weights()
sum(weights)

[0.33434001968876803, 0.16310034358009995]


0.497440363268868

In [79]:
# pick 1 random point and determine if it's above or below the line
x_coord = random.random() * 1000
y_coord = random.random() * 1000
line_y = line(x_coord)

print(x_coord, y_coord)
print(x_coord, line_y)

if y_coord > line_y:
    answer = 1
else:
    answer = 0
    
print(answer)

901.9672274233975 255.103627952859
901.9672274233975 450.98361371169875
0


In [80]:
# pass the random point into the node, feed it back and see if the weights change
guess = p.feed_forward([x_coord, y_coord])
p.backward_pass(guess - answer)
p.get_weights()

[-8.685332254545209, -2.38793593594849]


[-8.685332254545209, -2.38793593594849]

In [81]:
# train the network
for _ in range(0, 1000000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-136.2615564737694, 273.7106305630853]


[-136.2615564737694, 273.7106305630853]

In [82]:
# check the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (above == True and guess_above >= 0.5):
        correct += 1
    if (above == False and guess_above < 0.5):
        correct += 1

print(correct)

1000


Add bias to the node in order to handle lines that do not go through the origin.

In [4]:
def line(x):
    return 0.5 * x + 100

In [83]:
p = Perceptron(3)
weights = p.get_weights()

[-0.6539359462642047, 0.3223389234499692, -0.9107349737727213]


In [84]:
# train the Perceptron using a bias term
for _ in range(0, 1000000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    if y_coord > line_y:
        answer = 1
    else:
        answer = 0
    
    guess = p.feed_forward([x_coord, y_coord, 1])
    p.backward_pass(guess - answer)
    
p.get_weights()

[-140.40179768225235, 281.15690280822787, -0.8807349737727213]


[-140.40179768225235, 281.15690280822787, -0.8807349737727213]

In [85]:
# determine the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    line_y = line(x_coord)
    
    above = y_coord > line_y
    guess_above = p.feed_forward([x_coord, y_coord, 1])
    
    if (above == True and guess_above >= 0.5):
        correct += 1
    if (above == False and guess_above < 0.5):
        correct += 1

print(correct)

999


**Perceptrons for Logic**

Implement logicial AND and OR

In [218]:
a = Perceptron(3, act='step') # logical AND
b = Perceptron(3, act='step') # logical OR

[0.038518347812716014, 0.6911857076159746, -0.10595426553480292]
[0.2220752096841767, -0.18772657089782818, 0.765483827757995]


In [219]:
# Train the 'a' Perceptron for logical AND
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
#     print(first, second, first and second)
    a_out = a.feed_forward([first, second, 1])
    if (first and second):
        answer = 1
    else:
        answer = 0
    a.backward_pass(a_out - answer)
    
print(a.get_weights())

# Train the 'b' Perceptron for logical OR
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
#     print(first, second, first and second)
    b_out = b.feed_forward([first, second, 1])
    if (first or second):
        answer = 1
    else:
        answer = 0
    b.backward_pass(b_out - answer)
    
print(b.get_weights())

[0.038518347812716014, 0.3911857076159744, -0.4059542655348031]
[0.2220752096841767, 0.012273429102171851, -0.004516172242005597]


In [220]:
print("Logical AND")
print(a.feed_forward([1, 1, 1]))
print(a.feed_forward([1, 0, 1]))
print(a.feed_forward([0, 1, 1]))
print(a.feed_forward([0, 0, 1]))
print("\nLogical OR")
print(b.feed_forward([1, 1, 1]))
print(b.feed_forward([1, 0, 1]))
print(b.feed_forward([0, 1, 1]))
print(b.feed_forward([0, 0, 1]))

Logical AND
1
0
0
0

Logical OR
1
1
1
0


**XOR with 2 Perceptrons**

In [136]:
a = Perceptron(3)
b = Perceptron(4)

[0.8988586797569746, -0.13590476509619154, 0.46911879534340906]
[-0.6060649979570265, -0.6788434420222427, 0.8627717347303085, 0.5938530214547715]


In [137]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, a_out, second, 1])
    return b_out

In [138]:
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, a_out, second, 1])
    
    if (first != second):
        answer = 1
    else:
        answer = 0
        
    back_error = b.backward_pass(b_out - answer)
    a.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())

[8.276476689803884, -7.473667151086369, 3.815978113044765]
[6.273814226610519, -12.756245291974228, -5.9167224170567385, 9.323145785008206]


In [139]:
print('%f' % network(1, 1))
print('%f' % network(1, 0))
print('%f' % network(0, 1))
print('%f' % network(0, 0))

0.049670
0.944840
0.956298
0.040765


**XOR Network with 3 Perceptrons**

In [145]:
a = Perceptron(3)
b = Perceptron(3)
c = Perceptron(3)

[0.5134863058498789, -0.05206475037782021, -0.4124302451197219]
[0.2515330576615058, -0.05293676830790184, -0.9138025216811596]
[-0.7256076477312332, -0.05931151418978131, -0.11221658078767072]


In [146]:
def network(first, second):
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, second, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    return c_out

In [147]:
for _ in range(0,1000000):
    first = random.choice([0, 1])
    second = random.choice([0, 1])
    
    a_out = a.feed_forward([first, second, 1])
    b_out = b.feed_forward([first, second, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    
    if (first != second):
        answer = 1
    else:
        answer = 0
        
    back_error = c.backward_pass(c_out - answer)
    a.backward_pass(back_error[0])
    b.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())
print(c.get_weights())

[5.5125182918336, -5.251054497177363, 2.610426424601591]
[5.849633684268684, -5.82832150693468, -3.24842384281161]
[-8.187719783749245, 8.451382211452637, 3.8563290534347705]


In [148]:
print('%f' % network(1, 1))
print('%f' % network(1, 0))
print('%f' % network(0, 1))
print('%f' % network(0, 0))

0.027375
0.971771
0.964847
0.030625


In [149]:
print('%f' % a.feed_forward([1, 1, 1]))
print('%f' % a.feed_forward([1, 0, 1]))
print('%f' % a.feed_forward([0, 1, 1]))
print('%f' % a.feed_forward([0, 0, 1]))
print("\n")
print('%f' % b.feed_forward([1, 1, 1]))
print('%f' % b.feed_forward([1, 0, 1]))
print('%f' % b.feed_forward([0, 1, 1]))
print('%f' % b.feed_forward([0, 0, 1]))
print("\n")
print('%f' % c.feed_forward([1, 1, 1]))
print('%f' % c.feed_forward([1, 0, 1]))
print('%f' % c.feed_forward([0, 1, 1]))
print('%f' % c.feed_forward([0, 0, 1]))

0.946439
0.999703
0.066569
0.931530


0.038158
0.930939
0.000114
0.037384


0.984015
0.012979
0.999995
0.979292


**Parabola**

Utilizing non-lenear activation

In [150]:
def parabola(x):
    return 0.005 * pow(x - 500, 2) + 250

In [210]:
a = Perceptron(3, act='step')
b = Perceptron(3, act='step')
c = Perceptron(3, act='step')
# a = Perceptron(3)
# b = Perceptron(3)
# c = Perceptron(3)

[-0.15499689781106518, 0.2272443826066548, 0.4216641146819553]
[0.7340591592212253, 0.6116375015720088, 0.5361495461216452]
[-0.9885759854543825, 0.4832936822788094, -0.2977654632988376]


In [216]:
for _ in range(0,1000000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    curve_y = parabola(x_coord)
    x_norm = x_coord / 1000
    y_norm = y_coord / 1000
        
    a_out = a.feed_forward([x_norm, y_norm, 1])
    b_out = b.feed_forward([x_norm, y_norm, 1])
    c_out = c.feed_forward([a_out, b_out, 1])
    
    if curve_y > y_coord:
        answer = 1
    else:
        answer = 0

    back_error = c.backward_pass(c_out - answer)
    a.backward_pass(back_error[0])
    b.backward_pass(back_error[1])
    
print(a.get_weights())
print(b.get_weights())
print(c.get_weights())

[-14.796762807734593, 414.31719183198544, -61.63191027470704]
[-0.23111027410754464, -18.731829386152757, 0.3040904396026934]
[-0.33857598545438194, 0.013293682278809138, 0.3522345367011627]


In [217]:
# determine the accuracy
correct = 0

for _ in range(0,1000):
    x_coord = random.random() * 1000
    y_coord = random.random() * 1000
    curve_y = parabola(x_coord)
    x_norm = x_coord / 1000
    y_norm = y_coord / 1000
    
    above = curve_y > y_coord
    guess_above = network(x_norm, y_norm)
    
    if (above == True and guess_above >= 0.5):
        correct += 1
    if (above == False and guess_above < 0.5):
        correct += 1

print(correct)

602
